# Contents
* [Introduction](#Introduction)
* [Imports and configuration](#Imports-and-configuration)
* [Load data](#Load-data)
* [Setup](#Setup)
* [Evaluate](#Evaluate)
  * [By speaker gender](#By-speaker-gender)
  * [By language](#By-language)
  * [By valence (OvR)](#By-valence-(OvR))
  * [One English sample](#One-English-sample)
* [Results](#Results)
* [Discussion](#Discussion)

# Introduction

We have now trained 4 prototypes on features engineered from FRILL embeddings. In addition, we have extracted the same features from holdout data from three new sources. In this notebook, we evaluate the classification performance of the prototypes on the unseen data.

# Imports and configuration

In [ ]:
from time import time

notebook_begin_time = time()

# set random seeds

from os import environ
from random import seed as random_seed
from numpy.random import seed as np_seed
from tensorflow.random import set_seed


def reset_seeds(seed: int) -> None:
    """Utility function for resetting random seeds"""
    environ["PYTHONHASHSEED"] = str(seed)
    random_seed(seed)
    np_seed(seed)
    set_seed(seed)


reset_seeds(SEED := 2021)

In [ ]:
# extensions
%load_ext autotime
%load_ext lab_black
%load_ext nb_black

In [ ]:
# core
import numpy as np
import pandas as pd

# utility
from gc import collect as gc_collect
from joblib import dump, load

from tqdm.notebook import tqdm

# typing
from typing import Callable, Dict, List

# faster sklearn
from sklearnex import patch_sklearn

patch_sklearn()
del patch_sklearn

# metrics
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import (
    accuracy_score,
    average_precision_score,
    balanced_accuracy_score,
    brier_score_loss,
    f1_score,
    log_loss,
    precision_score,
    recall_score,
    roc_auc_score,
    top_k_accuracy_score,
)

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context("notebook")
%matplotlib inline

# full display of rows and columns
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)

# display outputs w/o print calls
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
del InteractiveShell

In [ ]:
# Location of holdout data
HOLDOUT_DATA = "."

# Location of holdout labels
HOLDOUT_LABELS = "."

# Location where this notebook will output
OUT_FOLDER = "."

# Location of prototype joblib files
PROTOTYPES = "./prototypes"

_ = gc_collect()

# Load data

In [ ]:
data = pd.read_feather(f"{HOLDOUT_DATA}/holdout_featurized.feather")
labels = pd.read_feather(f"{HOLDOUT_LABELS}/holdout_labels.feather")
labels = labels.loc[labels.emo != "sur"]
labels = labels.loc[data.index]
assert len(labels) == len(data)
assert all(labels.index == data.index)

y_true = labels.valence

# Setup

In this section, we define some structures to keep track of models and scores.

In [ ]:
MODELS = (
    "bagging_GNB",
    "logreg",
    "ridge",
    "stacked_gnb_ridge",
    "stacked_gnb_ridge_passthrough",
    "voting_gnb_ridge",
)
load_model = lambda model: load(f"{PROTOTYPES}/{model}.joblib")

# ternary to binary labels per class
RECODER = {
    "neg": lambda y: ((y - 1) // 2) * (-1),
    "neu": lambda y: y % 2,
    "pos": lambda y: y // 2,
}

VALENCE = {"neg": 0, "neu": 1, "pos": 2}


def binary_scorer(scorer: Callable, valence: str) -> Callable:
    "Returns a binary scorer for the given class (valence)"

    def new_scorer(y_true, y_score) -> float:
        y_true = RECODER[valence](y_true)
        y_score = RECODER[valence](np.squeeze(y_score))
        try:  # for predict_proba output
            _ = len(y_score[0])  # len(int) should throw a TypeError
            y_score: np.ndarray = y_score[:, VALENCE[valence]]
        except TypeError:
            y_score = RECODER[valence](y_score)
        return scorer(y_true, y_score)

    return new_scorer


METRICS = {
    "accuracy": accuracy_score,
    "adjusted_balanced_accuracy": lambda y_true, y_pred: balanced_accuracy_score(
        y_true, y_pred, adjusted=True
    ),
    "balanced_accuracy": balanced_accuracy_score,  # default
    "macro_f1": lambda y_true, y_pred: f1_score(y_true, y_pred, average="macro"),
    "micro_f1": lambda y_true, y_pred: f1_score(y_true, y_pred, average="micro"),
    "weighted_f1": lambda y_true, y_pred: f1_score(y_true, y_pred, average="weighted"),
    "macro_precision": lambda y_true, y_pred: precision_score(
        y_true, y_pred, average="macro"
    ),
    "micro_precision": lambda y_true, y_pred: precision_score(
        y_true, y_pred, average="micro"
    ),
    "weighted_precision": lambda y_true, y_pred: precision_score(
        y_true, y_pred, average="weighted"
    ),
    "macro_recall": lambda y_true, y_pred: recall_score(
        y_true, y_pred, average="macro"
    ),
    "micro_recall": lambda y_true, y_pred: recall_score(
        y_true, y_pred, average="micro"
    ),
    "weighted_recall": lambda y_true, y_pred: recall_score(
        y_true, y_pred, average="weighted"
    ),
    "geometric_mean": lambda y_true, y_pred: geometric_mean_score(
        y_true, y_pred, average="multiclass"
    ),
    "macro_geometric_mean": lambda y_true, y_pred: geometric_mean_score(
        y_true, y_pred, average="macro"
    ),
    "micro_geometric_mean": lambda y_true, y_pred: geometric_mean_score(
        y_true, y_pred, average="micro"
    ),
    # the below require .predict_proba output
    "auroc_macro_ovo": lambda y_true, y_score: roc_auc_score(
        y_true, y_score, average="macro", multi_class="ovo"
    ),
    "auroc_macro_ovr": lambda y_true, y_score: roc_auc_score(
        y_true, y_score, average="macro", multi_class="ovr"
    ),
    "auroc_weighted_ovo": lambda y_true, y_score: roc_auc_score(
        y_true, y_score, average="weighted", multi_class="ovo"
    ),
    "auroc_weighted_ovr": lambda y_true, y_score: roc_auc_score(
        y_true, y_score, average="weighted", multi_class="ovr"
    ),
    "log_loss": log_loss,
    "top_2_accuracy": top_k_accuracy_score,
    # the below must be adapted to multiclass and require .predict_proba output
    "average_precision_neg": binary_scorer(
        lambda y_true, y_score: average_precision_score(
            y_true, y_score, average="weighted"
        ),
        "neg",
    ),
    "average_precision_neu": binary_scorer(
        lambda y_true, y_score: average_precision_score(
            y_true, y_score, average="weighted"
        ),
        "neu",
    ),
    "average_precision_pos": binary_scorer(
        lambda y_true, y_score: average_precision_score(
            y_true, y_score, average="weighted"
        ),
        "pos",
    ),
    "brier_loss_neg": binary_scorer(brier_score_loss, "neg"),
    "brier_loss_neu": binary_scorer(brier_score_loss, "neu"),
    "brier_loss_pos": binary_scorer(brier_score_loss, "pos"),
}
PREDICT_PROBA_KEYS = {
    "auroc_macro_ovo",
    "auroc_macro_ovr",
    "auroc_weighted_ovo",
    "auroc_weighted_ovr",
    "log_loss",
    "top_2_accuracy",
    "average_precision_neg",
    "average_precision_neu",
    "average_precision_pos",
    "brier_loss_neg",
    "brier_loss_neu",
    "brier_loss_pos",
}

results = {
    "model": [],
    "score_set": [],
    **{metric: [] for metric in METRICS.keys()},
}

GNB_FEATURES = ["spherical-LDA1", "spherical-LDA2"]


def score_model(
    model_name: str, score_set: str, data: pd.DataFrame, results: Dict[str, List]
) -> Dict[str, List]:
    """Score a fitted model using METRICS and record in results"""
    results["model"].append(model_name)
    results["score_set"].append(score_set)
    model = load_model(model_name)
    df = data
    if model_name == "bagging_GNB":
        df = data.loc[:, GNB_FEATURES]
    _ = gc_collect()
    y_true_ = labels.loc[df.index, "valence"]
    y_pred = model.predict(df)
    y_prob = model.predict_proba(df)
    dump(y_prob, f"{OUT_FOLDER}/predictions/{model_name}_{score_set}.pkl")
    for metric, scorer in tqdm(METRICS.items()):
        try:
            results[metric].append(
                scorer(y_true_, y_prob if metric in PREDICT_PROBA_KEYS else y_pred)
            )
        except ValueError as e:
            print(e)
            results[metric].append(np.nan)
    return results


_ = gc_collect()

# Evaluate

In [ ]:
for model in tqdm(MODELS):
    results = score_model(model, "ternary", data, results)

In [ ]:
results_df = pd.DataFrame(results)
for metric in METRICS.keys():
    print(
        results_df.loc[
            results_df[metric]
            == (
                results_df[metric].min()
                if "loss" in metric
                else results_df[metric].max()
            ),
            ["model", metric],
        ]
    )

In [ ]:
# visualize
for metric in METRICS.keys():
    df = results_df.sort_values(by=metric, ascending="loss" in metric)
    g = sns.catplot(
        x=metric,
        y="model",
        data=df,
        kind="bar",
        linewidth=0,
        palette="colorblind",
    )
    ax = plt.gca()
    _ = ax.bar_label(ax.containers[0])
    sns.despine(left=True, top=True, right=True)
    _ = plt.ylabel("")
    plt.tight_layout()
    plt.show()
    del df
    del g
    del ax
    _ = gc_collect()

## By speaker gender

In [ ]:
for model in tqdm(MODELS):
    results = score_model(
        model, "m_speaker", data.loc[labels.speaker_gender == "m", :], results
    )
for model in tqdm(MODELS):
    results = score_model(
        model, "f_speaker", data.loc[labels.speaker_gender == "f", :], results
    )

In [ ]:
results_df = pd.DataFrame(results)
results_df = results_df.loc[results_df.score_set.isin({"f_speaker", "m_speaker"}), :]
for metric in METRICS.keys():
    df = results_df.sort_values(by=metric, ascending="loss" in metric)
    g = sns.catplot(
        x="model",
        y=metric,
        data=df,
        kind="bar",
        hue="score_set",
        hue_order=["f_speaker", "m_speaker"],
        palette="colorblind",
        linewidth=0,
        height=4,
        aspect=3,
        legend=False,
    )
    sns.despine(top=True, bottom=True, right=True)
    _ = plt.xlabel("")
    _ = plt.ylabel("")
    _ = plt.suptitle(metric, y=1.1)
    _ = plt.legend(
        bbox_to_anchor=(0.5, 1.15), loc="upper center", ncol=2, frameon=False
    )
    ax = plt.gca()
    for container in ax.containers:
        _ = ax.bar_label(container)
    plt.show()
    del df
    del g
    del ax
    _ = gc_collect()

In contrast to the first prototypes, it looks like the new prototypes are better with male speakers by nearly all metrics.

## By language

This is roughly equivalent to the data source of the holdout sets. Note that x4nth055_SER_custom contains one sample in English, which is omitted in this section's analysis.

In [ ]:
for model in tqdm(MODELS):
    results = score_model(model, "arb", data.loc[labels.lang1 == "arb", :], results)
for model in tqdm(MODELS):
    results = score_model(model, "deu", data.loc[labels.lang1 == "deu", :], results)
for model in tqdm(MODELS):
    results = score_model(model, "kor", data.loc[labels.lang1 == "kor", :], results)

The Arabic dataset only has two emotions (happy and neutral).

In [ ]:
results_df = pd.DataFrame(results)
results_df = results_df.loc[results_df.score_set.isin({"arb", "deu", "kor"}), :]
for metric in METRICS.keys():
    df = results_df.sort_values(by=metric, ascending="loss" in metric)
    g = sns.catplot(
        x="model",
        y=metric,
        data=df,
        kind="bar",
        hue="score_set",
        hue_order=["arb", "deu", "kor"],
        palette="colorblind",
        linewidth=0,
        height=4,
        aspect=3,
        legend=False,
    )
    sns.despine(top=True, bottom=True, right=True)
    _ = plt.xlabel("")
    _ = plt.ylabel("")
    _ = plt.suptitle(metric, y=1.1)
    plt.legend(bbox_to_anchor=(0.5, 1.15), loc="upper center", ncol=3, frameon=False)
    ax = plt.gca()
    for container in ax.containers:
        _ = ax.bar_label(container)
    plt.show()
    del df
    del g
    del ax
    _ = gc_collect()

## By valence (OvR)

In [ ]:
for valence in ("neg", "neu", "pos"):
    for model in tqdm(MODELS):
        results["model"].append(model)
        results["score_set"].append(valence)
        _ = model
        df = data.loc[:, GNB_FEATURES] if model == "bagging_GNB" else data
        model = load_model(model)
        y_pred = model.predict(df)
        y_prob = model.predict_proba(df)
        dump(y_prob, f"{OUT_FOLDER}/predictions/{_}_{valence}.pkl")
        del df
        del model
        _ = gc_collect()
        for metric, scorer in tqdm(METRICS.items()):
            if "average_precision" in metric or "brier_score" in metric:
                if metric[-3:] != valence:
                    results[metric].append(np.nan)
                    continue
            results[metric].append(
                binary_scorer(scorer, valence)(
                    y_true, y_prob if metric in PREDICT_PROBA_KEYS else y_pred
                )
            )

In [ ]:
results_df = pd.DataFrame(results)
results_df = results_df.loc[
    results_df.score_set.isin({"ternary", "neg", "neu", "pos"}), :
]
for metric in METRICS.keys():
    df = results_df.sort_values(by=metric, ascending="loss" in metric)
    g = sns.catplot(
        x="model",
        y=metric,
        data=df,
        kind="bar",
        hue="score_set",
        hue_order=["ternary", "neg", "neu", "pos"],
        palette="colorblind",
        linewidth=0,
        height=4,
        aspect=3,
        legend=False,
    )
    sns.despine(top=True, bottom=True, right=True)
    _ = plt.xlabel("")
    _ = plt.ylabel("")
    _ = plt.suptitle(metric, y=1.1)
    _ = plt.legend(
        bbox_to_anchor=(0.5, 1.15), loc="upper center", ncol=4, frameon=False
    )
    ax = plt.gca()
    for container in ax.containers:
        _ = ax.bar_label(container)
    plt.show()
    del df
    del g
    del ax
    _ = gc_collect()

# One English sample

This is a positive sample, so 2 would be a correct prediction.

In [ ]:
df_ = data.loc[labels.lang1 == "eng", :]
for model in MODELS:
    df = df_
    if "GNB" in model:
        df = df_.loc[:, GNB_FEATURES]
    print(f"{model} predicted {load_model(model).predict(df)}")
del df
_ = gc_collect()
# correct prediction is 2

In [ ]:
results_df = pd.DataFrame(results)
results_df.columns = results_df.columns.astype(str)
results_df.reset_index(drop=True).to_csv(f"{OUT_FOLDER}/holdout_results.csv")

# Discussion

How surprising to see such poor performance distinguishing negative samples.

Overall, I am disappointed. There is a lot of room for improvement and many new approaches we could take with the data:
* resampling
* augmentation
* more FRILL-based engineering
* low-level audio features
* spectrograms/MFCCs
* deep learning

* best AUROC: 64.1% (ridge)
* best accuracy: 50.7% (bagging_GNB)
* best log loss: 1.139 (voting_gnb_ridge)

I'll probably use the voting ensemble of bagging GNB and ridge classifier since it has the lowest loss.

In [ ]:
print(f"Time elapsed since notebook_begin_time: {time() - notebook_begin_time} s")
_ = gc_collect()

[^top](#Contents)